In [1]:
import os
import json
import requests # pipenv install requests
import random
#import time
from django.core.files.base import ContentFile
from shop.models import Category, Shop, Item
from urllib.parse import urljoin
from fake_useragent import UserAgent # pipenv install fake-useragent

In [2]:
class Yogiyo:
    HOST = 'https://www.yogiyo.co.kr'
    
    def __init__(self, headers):
        self.headers = headers
    
    def get_restaurant_list_by_geo(self, zip_code):
        url = self.HOST + '/api/v1/restaurants-geo/'
        params = dict(
                items=20,
                order='rank',
                page=0,
                search='',
                zip_code = zip_code
        )
        res = requests.get(url,params=params, headers = self.headers)
        return res.json()

    def get_menu_list(self, restaurant_id):
        base_url = self.HOST + '/api/v1/restaurants/{restaurant_id}/menu/?add_photo_menu=original'
        url = base_url.format(restaurant_id=restaurant_id)
        res = requests.get(url, headers=self.headers)
        return res.json()

In [3]:
user_agent = UserAgent()

In [4]:
headers = {
    'X-ApiKey' : 'iphoneap',
    'X-ApiSecret' : 'fe5183cc3dea12bd0ce299cf110a75a2',
    'X-MOD-SBB-CTYPE': 'xhr'
}

In [24]:
headers

{'X-ApiKey': 'iphoneap',
 'X-ApiSecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
 'X-MOD-SBB-CTYPE': 'xhr'}

In [5]:
yogiyo = Yogiyo(headers)

In [6]:
zip_code = '121070'
restaurants = yogiyo.get_restaurant_list_by_geo('121070')['restaurants']



In [7]:
for rest in restaurants:
    category_name = rest['categories'][0] # 요기요는 M:N 관계
    category = Category.objects.get(name=category_name)
    latlng = '{lat},{lng}'.format(**rest)
    logo_url = urljoin(Yogiyo.HOST, rest['logo_url'])
    shop = Shop(category=category, name=rest['name'], latlng=latlng, meta=rest)
    
    logo_name = os.path.basename(logo_url)
    logo_data = requests.get(logo_url).content
    shop.photo.save(logo_name, ContentFile(logo_data), save=False)
    print('{name}:{lat},{lng}, {categories}, {logo_url}'.format(**rest))
    
    shop.save()

페리카나-신수점:37.5430186999385,126.9369214, ['야식', '치킨', '프랜차이즈'], /media/restaurant_logos/페리카나_20160321_Franchise이미지약정서_crop_200x200_6Q610FD.jpg
김준현의피자헤븐-마포점:37.5535637,126.9456872, ['피자양식', '프랜차이즈'], /media/restaurant_logos/피자헤븐_20161011_Franchise이미지약정서_crop_200x200.jpg
돈까스퐁당떡볶이공수간-신촌직영점:37.5577122999405,126.9366662, ['분식', '일식돈까스', '한식', '프랜차이즈'], /media/franchise_logos/돈까스퐁당떡볶이공수간_20180605_Franchise이미지약정서_crop_200x200.jpg
원할머니보쌈-마포공덕점:37.5498578999394,126.9544187, ['프랜차이즈', '족발보쌈', '한식', '야식'], /media/restaurant_logos/원할머니보쌈02_20150716_Franchise이미지약정서_crop_200x200_kvU9sSM.jpg
新북경:37.5459633292082,126.945252566397, ['1인분주문', '중식'], /media/restaurant_logos/짜장면01_20131128_FoodAD_crop_200x200_kKwHEnP.jpg
혼밥혼술-신촌점:37.5556092705005,126.940414980044, ['한식', '야식', '일식돈까스', '1인분주문'], /media/restaurant_logos/혼밥혼술-신촌점_20180309_가맹로고이미지약정서_crop_200x200.JPG
파파이스-이대역점:37.5578173,126.9461084, ['피자양식', '치킨', '프랜차이즈', '1인분주문'], /media/restaurant_logos/파파이스_20170322_Franchise이미지약정서_crop_200x200_DV3Jhhq.jp

In [8]:
for shop in Shop.objects.all():
    restaurant_id = shop.meta['id'] # 요기요 내에서의 restaurant_id
    menu_list = yogiyo.get_menu_list(restaurant_id)
    for sub_menu_list in menu_list:
        items = sub_menu_list['items']
        for item_meta in items:
            item = Item(shop=shop, name=item_meta['name'], amount=item_meta['price'], meta=item_meta, rating=1)
            item_image_url = item_meta.get('image', '') # image 없는 메뉴도 있음
            if item_image_url:
                item_image_url = urljoin(Yogiyo.HOST, item_meta['image'])
                item_image_name = os.path.basename(item_image_url)
                item_image_data = requests.get(item_image_url).content
                item.photo.save(item_image_name, ContentFile(item_image_data), save=False)
            
            print('saving item : {}'.format(item.name))
            print(item)
            item.save()
print('?')

saving item : [행사]새우천왕L
[행사]새우천왕L
saving item : [행사]치우천왕L
[행사]치우천왕L
saving item : [행사]씨푸드빠에야앤스테이크L
[행사]씨푸드빠에야앤스테이크L
saving item : [행사]하프앤하프L
[행사]하프앤하프L
saving item : [행사]쉬림프골드L
[행사]쉬림프골드L
saving item : [행사]따블퐈L
[행사]따블퐈L
saving item : [행사]하프앤하프L
[행사]하프앤하프L
saving item : [행사]따블퐈L
[행사]따블퐈L
saving item : [행사]씨푸드빠에야앤스테이크L
[행사]씨푸드빠에야앤스테이크L
saving item : 핫 윙（8조각）
핫 윙（8조각）
saving item : 하와이안 딜라이트R
하와이안 딜라이트R
saving item : [행사]쉬림프골드L
[행사]쉬림프골드L
saving item : 오븐치즈 미트스파게티
오븐치즈 미트스파게티
saving item : [행사]포테이토골드L
[행사]포테이토골드L
saving item : 오븐치즈 베이컨 크림 스파게티
오븐치즈 베이컨 크림 스파게티
saving item : 페퍼로니R
페퍼로니R
saving item : 새우천왕R
새우천왕R
saving item : 치우천왕R
치우천왕R
saving item : 씨푸드빠에야앤스테이크R
씨푸드빠에야앤스테이크R
saving item : 씨푸드빠에야R
씨푸드빠에야R
saving item : 따블퐈R
따블퐈R
saving item : 트레비앙R
트레비앙R
saving item : 쉬림프골드R
쉬림프골드R
saving item : 로맨틱콤보R
로맨틱콤보R
saving item : 러블리피스R
러블리피스R
saving item : 스윗피스R
스윗피스R
saving item : 시크릿가든R
시크릿가든R
saving item : 포테이토골드R
포테이토골드R
saving item : 하프앤하프R （쉬림프골드 R반 ＋포테이토 골드 R반）
하프앤하프R （쉬림프골드 R반 ＋포테이토 골드 

saving item : 치킨 수퍼파파스 골드링 크러스트 L
치킨 수퍼파파스 골드링 크러스트 L
saving item : 멜로우 머쉬룸 골드링 크러스트 L
멜로우 머쉬룸 골드링 크러스트 L
saving item : 식스 치즈 골드링 크러스트 L
식스 치즈 골드링 크러스트 L
saving item : 스파이시 이탈리안 골드링 크러스트 L
스파이시 이탈리안 골드링 크러스트 L
saving item : 마가리타 골드링 크러스트 L
마가리타 골드링 크러스트 L
saving item : 페퍼로니 골드링 크러스트 L
페퍼로니 골드링 크러스트 L
saving item : 하와이안 골드링 크러스트 L
하와이안 골드링 크러스트 L
saving item : 가든 스페셜 골드링 크러스트 L
가든 스페셜 골드링 크러스트 L
saving item : 핫 앤 스파이시 멕시칸 골드링 크러스트 L
핫 앤 스파이시 멕시칸 골드링 크러스트 L
saving item : 핫 치킨 바베큐 골드링 크러스트 L
핫 치킨 바베큐 골드링 크러스트 L
saving item : 치킨 바베큐 골드링 크러스트 F
치킨 바베큐 골드링 크러스트 F
saving item : 수퍼 파파스 골드링 크러스트 F
수퍼 파파스 골드링 크러스트 F
saving item : 아이리쉬 포테이토 골드링 크러스트 F
아이리쉬 포테이토 골드링 크러스트 F
saving item : 스파이시 치킨 랜치 골드링 크러스트 F
스파이시 치킨 랜치 골드링 크러스트 F
saving item : 존스 페이버릿 골드링 크러스트 F
존스 페이버릿 골드링 크러스트 F
saving item : 더블 치즈버거 골드링 크러스트 F
더블 치즈버거 골드링 크러스트 F
saving item : 프리미엄 직화불고기 골드링 크러스트 F
프리미엄 직화불고기 골드링 크러스트 F
saving item : 올미트 골드링 크러스트 F
올미트 골드링 크러스트 F
saving item : 치킨 수퍼파파스 골드링 크러스트 F
치킨 수퍼파파스 골드링 크러스트 F
saving item 

saving item : 더블퐁듀 쉬림프 （30% 할인가）M
더블퐁듀 쉬림프 （30% 할인가）M
saving item : 크런치 큐브 스테이크M （30% 할인가）
크런치 큐브 스테이크M （30% 할인가）
saving item : 크런치 치즈 스테이크 M （30% 할인금액）
크런치 치즈 스테이크 M （30% 할인금액）
saving item : 크런치 토핑킹M （30% 할인가）
크런치 토핑킹M （30% 할인가）
saving item : 크런치 갈릭버터 쉬림프M （30% 할인가）
크런치 갈릭버터 쉬림프M （30% 할인가）
saving item : 크런치 수퍼슈프림M （30% 할인가）
크런치 수퍼슈프림M （30% 할인가）
saving item : 더블박스
더블박스
saving item : 리치치즈 파스타
리치치즈 파스타
saving item : 크런치 큐브 스테이크M （30% 할인가）
크런치 큐브 스테이크M （30% 할인가）
saving item : 더블퐁듀 쉬림프L 세트
더블퐁듀 쉬림프L 세트
saving item : 더블퐁듀 쉬림프 （30% 할인가）M
더블퐁듀 쉬림프 （30% 할인가）M
saving item : 갈릭버터 쉬림프M （30% 할인가）
갈릭버터 쉬림프M （30% 할인가）
saving item : 크런치 큐브 스테이크L （30% 할인가）
크런치 큐브 스테이크L （30% 할인가）
saving item : 팬피자 - 치즈M
팬피자 - 치즈M
saving item : 더블퐁듀 쉬림프 （30% 할인가）L
더블퐁듀 쉬림프 （30% 할인가）L
saving item : 베이컨 포테이토L （30% 할인가）
베이컨 포테이토L （30% 할인가）
saving item : 더블퐁듀 쉬림프L 세트
더블퐁듀 쉬림프L 세트
saving item : 더블퐁듀 쉬림프 （30% 할인가）M
더블퐁듀 쉬림프 （30% 할인가）M
saving item : 더블퐁듀 쉬림프 （30% 할인가）L
더블퐁듀 쉬림프 （30% 할인가）L
saving item : 크런치 큐브 스테이크M （30% 할인가）
크

saving item : 더블 치즈 BIG L
더블 치즈 BIG L
saving item : 페퍼로니 BIG L
페퍼로니 BIG L
saving item : 하와이안 BIG L
하와이안 BIG L
saving item : 아메리칸 스페셜 BIG L
아메리칸 스페셜 BIG L
saving item : 더블 치즈 베이컨 BIG L
더블 치즈 베이컨 BIG L
saving item : 헤븐스페셜 BIG L
헤븐스페셜 BIG L
saving item : 슈퍼 슈프림 BIG L
슈퍼 슈프림 BIG L
saving item : 고구마 스위트 BIG L
고구마 스위트 BIG L
saving item : 포테이토 BIG L
포테이토 BIG L
saving item : 핫 앤 스파이시 BIG L
핫 앤 스파이시 BIG L
saving item : 불고기 BIG L
불고기 BIG L
saving item : 더블 포테이토 BIG L
더블 포테이토 BIG L
saving item : 레드 핫 치킨 BIG L
레드 핫 치킨 BIG L
saving item : 쉬림프 디럭스 BIG L
쉬림프 디럭스 BIG L
saving item : 치킨 바베큐 BIG L
치킨 바베큐 BIG L
saving item : 쿼터 드레싱 포테이토 BIG L
쿼터 드레싱 포테이토 BIG L
saving item : 헤븐리 4치즈 BIG L
헤븐리 4치즈 BIG L
saving item : 그릴드 포크 벨리 BIG L
그릴드 포크 벨리 BIG L
saving item : 크랜베리 고르곤졸라 BIG L
크랜베리 고르곤졸라 BIG L
saving item : 미트 킹덤 BIG L
미트 킹덤 BIG L
saving item : 눈꽃 치즈 스테이크 BIG L
눈꽃 치즈 스테이크 BIG L
saving item : 로얄 쿼터블 BIG L
로얄 쿼터블 BIG L
saving item : 고구마 스위트 BL（하프） ＋ BL（하프） 선택
고구마 스위트 BL（하프） ＋ BL（하프） 선택
saving item : 그릴드 포크

saving item : 반반（와사비톡＋마늘치킨）
반반（와사비톡＋마늘치킨）
saving item : 반반（매운양념＋양념치킨）
반반（매운양념＋양념치킨）
saving item : 반반（매운후라이드＋간장치킨）
반반（매운후라이드＋간장치킨）
saving item : 순살반반（양념치킨＋후라이드）
순살반반（양념치킨＋후라이드）
saving item : 순살반반（매운양념＋후라이드）
순살반반（매운양념＋후라이드）
saving item : 순살반반（간장치킨＋양념치킨）
순살반반（간장치킨＋양념치킨）
saving item : 순살반반（양념치킨＋매운양념）
순살반반（양념치킨＋매운양념）
saving item : 순살반반（간장치킨＋후라이드）
순살반반（간장치킨＋후라이드）
saving item : 순살반반（간장치킨＋매운양념）
순살반반（간장치킨＋매운양념）
saving item : 순살반반（간장치킨＋마늘치킨）
순살반반（간장치킨＋마늘치킨）
saving item : 순살반반（마늘치킨＋양념치킨）
순살반반（마늘치킨＋양념치킨）
saving item : 순살반반（매운후라이드＋양념치킨）
순살반반（매운후라이드＋양념치킨）
saving item : 순살반반（매운양념＋마늘치킨）
순살반반（매운양념＋마늘치킨）
saving item : 순살반반（마늘치킨＋후라이드）
순살반반（마늘치킨＋후라이드）
saving item : 순살반반（매운후라이드＋간장치킨）
순살반반（매운후라이드＋간장치킨）
saving item : 순살반반（매운후라이드＋매운양념）
순살반반（매운후라이드＋매운양념）
saving item : 순살반반（매운양념＋간장치킨）
순살반반（매운양념＋간장치킨）
saving item : 순살반반（치즈뿌리오＋양념치킨）
순살반반（치즈뿌리오＋양념치킨）
saving item : 순살반반（치즈뿌리오＋간장치킨）
순살반반（치즈뿌리오＋간장치킨）
saving item : 순살반반（치즈뿌리오＋후라이드）
순살반반（치즈뿌리오＋후라이드）
saving item : 순살반반（치즈뿌리오＋매운양념）
순살반반（치즈뿌리오＋매운양념）
saving item : 순살반반（핫데블

...

saving item : 제육볶음
제육볶음
saving item : 모듬 고로케 
모듬 고로케 
saving item : 핫 윙（8조각）
핫 윙（8조각）
saving item : 치즈야끼볼
치즈야끼볼
saving item : 옛날도시락
옛날도시락
saving item : 제육도시락
제육도시락
saving item : 계란찜
계란찜
saving item : 공기밥
공기밥
saving item : 웨지감자튀김
웨지감자튀김
saving item : 주먹밥
주먹밥
saving item : 깨밥
깨밥
saving item : 환타（파인애플 맛）500mL
환타（파인애플 맛）500mL
saving item : 사이다 500mL
사이다 500mL
saving item : 콜라 500mL
콜라 500mL
saving item : 케이준치킨팩
케이준치킨팩
saving item : 케이준익스트림치킨버거세트
케이준익스트림치킨버거세트
saving item : 치킨휠레버거세트
치킨휠레버거세트
saving item : 치킨8조각팩
치킨8조각팩
saving item : 치즈베이컨치킨버거세트
치즈베이컨치킨버거세트
saving item : 싱글세트
싱글세트
saving item : 치킨1조각
치킨1조각
saving item : 케이준익스트림치킨버거세트
케이준익스트림치킨버거세트
saving item : 싱글세트
싱글세트
saving item : 케이준윙2조각
케이준윙2조각
saving item : 치즈베이컨치킨버거세트
치즈베이컨치킨버거세트
saving item : 케이준후라이（L）
케이준후라이（L）
saving item : 콜라 1.25L PET
콜라 1.25L PET
saving item : 핑거휠레2조각
핑거휠레2조각
saving item : 케이준익스트림치킨버거
케이준익스트림치킨버거
saving item : 치킨휠레버거세트
치킨휠레버거세트
saving item : 치킨1조각
치킨1조각
saving item : 핑거휠레2조각
핑거휠레2조각
saving item : 케이준윙2조각
케이준윙2조

saving item : 핫갈비레오
핫갈비레오
saving item : 붐바스틱
붐바스틱
saving item : 스윗텐더
스윗텐더
saving item : 치레카
치레카
saving item : 순살치레카
순살치레카
saving item : 치레카날개¸
치레카날개¸
saving item : 치레카다리¸
치레카다리¸
saving item : 커리퀸
커리퀸
saving item : 순살커리퀸
순살커리퀸
saving item : 커리퀸날개¸
커리퀸날개¸
saving item : 커리퀸다리¸
커리퀸다리¸
saving item : 맵스터
맵스터
saving item : 순살맵스터
순살맵스터
saving item : 맵스터날개¸
맵스터날개¸
saving item : 맵스터다리¸
맵스터다리¸
saving item : 맛초킹（한마리）
맛초킹（한마리）
saving item : 순살맛초킹
순살맛초킹
saving item : 맛초킹날개¸
맛초킹날개¸
saving item : 맛초킹다리¸
맛초킹다리¸
saving item : 뿌링클
뿌링클
saving item : 뿌링클날개¸
뿌링클날개¸
saving item : 뿌링클다리¸
뿌링클다리¸
saving item : 뿌링클HOT
뿌링클HOT
saving item : 순살뿌링클
순살뿌링클
saving item : 순살뿌링클HOT
순살뿌링클HOT
saving item : 바삭클
바삭클
saving item : 순살바삭클
순살바삭클
saving item : 해바라기후라이드치킨
해바라기후라이드치킨
saving item : 후라이드 반／양념 반
후라이드 반／양념 반
saving item : 해바라기핫후라이드치킨
해바라기핫후라이드치킨
saving item : 해바라기닭다리후라이드치킨
해바라기닭다리후라이드치킨
saving item : 닭날개후라이드
닭날개후라이드
saving item : 양념치킨
양념치킨
saving item : 매운맛양념치킨
매운맛양념치킨
saving item : 간장골드한마리
간장골드한마리
saving item : 간장골드날개

saving item : 아삭킹새우＋후라이드치킨
아삭킹새우＋후라이드치킨
saving item : 후라이드치킨
후라이드치킨
saving item : 양념치킨
양념치킨
saving item : 마늘간장치킨
마늘간장치킨
saving item : 데리야끼치킨
데리야끼치킨
saving item : 땡초갈릭치킨
땡초갈릭치킨
saving item : 치즈스노우퀸
치즈스노우퀸
saving item : 커리양념치킨
커리양념치킨
saving item : 119불양념
119불양념
saving item : 오리지널 두마리（후라이드치킨＋후라이드치킨）
오리지널 두마리（후라이드치킨＋후라이드치킨）
saving item : 오리지널 두마리（양념치킨＋후라이드치킨）
오리지널 두마리（양념치킨＋후라이드치킨）
saving item : 오리지널 두마리（마늘간장치킨＋후라이드치킨）
오리지널 두마리（마늘간장치킨＋후라이드치킨）
saving item : 오리지널두마리（데리야끼치킨＋후라이드치킨）
오리지널두마리（데리야끼치킨＋후라이드치킨）
saving item : 오리지널두마리（양념치킨＋마늘간장치킨）
오리지널두마리（양념치킨＋마늘간장치킨）
saving item : 오리지널두마리（양념치킨＋데리야끼치킨）
오리지널두마리（양념치킨＋데리야끼치킨）
saving item : 오리지널두마리（마늘간장치킨＋데리야끼치킨）
오리지널두마리（마늘간장치킨＋데리야끼치킨）
saving item : 프리미엄 두마리（땡초갈릭치킨＋후라이드치킨）
프리미엄 두마리（땡초갈릭치킨＋후라이드치킨）
saving item : 프리미엄 두마리（치즈스노우퀸＋후라이드치킨）
프리미엄 두마리（치즈스노우퀸＋후라이드치킨）
saving item : 프리미엄 두마리（커리양념치킨＋후라이드치킨）
프리미엄 두마리（커리양념치킨＋후라이드치킨）
saving item : 119불양념＋후라이드치킨
119불양념＋후라이드치킨
saving item : 황금감자
황금감자
saving item : 치즈스틱 6개
치즈스틱 6개
saving item : 단호박피자L
단호박피자L
saving item : 쉬림

saving item : 짬뽕밥
짬뽕밥
saving item : 공기밥
공기밥
saving item : 탕볶밥
탕볶밥
saving item : 탕짬면
탕짬면
saving item : 탕짜면
탕짜면
saving item : 볶짬면
볶짬면
saving item : 볶짜면
볶짜면
saving item : 짬짜면
짬짜면
saving item : 고기 왕 만두
고기 왕 만두
saving item : 김치 왕 만두
김치 왕 만두
saving item : 물만두
물만두
saving item : 군만두
군만두
saving item : 꽃빵（6개）
꽃빵（6개）
saving item : 라조육
라조육
saving item : 유린기
유린기
saving item : 동파육
동파육
saving item : 오향장육
오향장육
saving item : 뼈뺀라조기
뼈뺀라조기
saving item : 뼈뺀깐풍기
뼈뺀깐풍기
saving item : 난자완스
난자완스
saving item : 깐풍육
깐풍육
saving item : 고추잡채（꽃빵）
고추잡채（꽃빵）
saving item : 사천탕수육
사천탕수육
saving item : 마파두부
마파두부
saving item : 잡채
잡채
saving item : 탕수육
탕수육
saving item : 해삼탕
해삼탕
saving item : 전가복
전가복
saving item : 송이전복
송이전복
saving item : 팔보채
팔보채
saving item : 잡탕
잡탕
saving item : 깐쇼중새우
깐쇼중새우
saving item : 깐풍중새우
깐풍중새우
saving item : 유산슬
유산슬
saving item : 삼선누룽지탕
삼선누룽지탕
saving item : 양장피
양장피
saving item : 오품냉채
오품냉채
saving item : 사품냉채
사품냉채
saving item : 삼품냉채
삼품냉채
saving item : 해파리냉채
해파리냉채
saving item : 차돌삼선술국
차돌삼선술국
saving item : 삼선술국
삼

saving item : 로만 씨푸드 R
로만 씨푸드 R
saving item : 로만 비프 R
로만 비프 R
saving item : 로만 콤비 R
로만 콤비 R
saving item : 로만 화이트 R
로만 화이트 R
saving item : 로만 스위트 R
로만 스위트 R
saving item : Best Set（피자R 선택＋파스타 선택＋수제 코울슬로＋콜라 1.25L）
Best Set（피자R 선택＋파스타 선택＋수제 코울슬로＋콜라 1.25L）
saving item : 스파이시윙（6조각）
스파이시윙（6조각）
saving item : 치즈 볼로네제 스파게티
치즈 볼로네제 스파게티
saving item : 페페로니 R
페페로니 R
saving item : 양념반달감자
양념반달감자
saving item : 크림 스파게티
크림 스파게티
saving item : 슈퍼 콤비네이션 R
슈퍼 콤비네이션 R
saving item : 치킨텐더（6조각）
치킨텐더（6조각）
saving item : 수제 코울슬로
수제 코울슬로
saving item : Best Set（피자L 선택＋파스타 선택＋수제 코울슬로＋콜라 1.25L）
Best Set（피자L 선택＋파스타 선택＋수제 코울슬로＋콜라 1.25L）
saving item : 로만 골드 R
로만 골드 R
saving item : 로만 씨푸드 R
로만 씨푸드 R
saving item : 로만 비프 R
로만 비프 R
saving item : 로만 콤비 R
로만 콤비 R
saving item : 로만 화이트 R
로만 화이트 R
saving item : 로만 스위트 R
로만 스위트 R
saving item : 로만 골드 L
로만 골드 L
saving item : 로만 씨푸드 L
로만 씨푸드 L
saving item : 로만 비프 L
로만 비프 L
saving item : 로만 콤비 L
로만 콤비 L
saving item : 로만 화이트 L
로만 화이트 L
saving item : 로만 스위트 L
로만 스위트 L
saving item : 프라임 